In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
%matplotlib inline
import cv2
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['labels.csv', 'train', 'test', 'sample_submission.csv']


In [10]:
from os.path import join
from sklearn.model_selection import train_test_split,StratifiedKFold
from tensorflow.python import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, Conv2D, Dropout, BatchNormalization,MaxPooling2D
from tensorflow.python.keras.preprocessing.image import load_img, img_to_array

In [3]:
labels = pd.read_csv("../input/labels.csv")
labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [4]:
#convert breed from categories to numbers
dogs=labels.breed.unique()

breeds={}

num=0
for item in dogs:
    breeds[item]=num
    num+=1
    
labels.breed=[breeds[item] for item in labels.breed]

In [5]:
#creating a dataframe with full Image Path
img_paths = [join("../input/train/", id+".jpg") for id in labels["id"]]

In [6]:
def read_and_prep_images(img_paths, img_height=100, img_width=100):
    imgs = [load_img(img_path, target_size=(img_height, img_width)) for img_path in img_paths]
    img_array = np.array([img_to_array(img)/255 for img in imgs])
    return img_array

In [7]:
train_data1 = read_and_prep_images(img_paths)

In [15]:
Y=labels["breed"]

In [8]:
out_y=keras.utils.to_categorical(labels["breed"])

In [12]:
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(train_data1, out_y, test_size=0.33, random_state=42)

In [ ]:
cvscores = []
H=[]
for train, test in kfold.split(train_data1, Y):
    model=Sequential()
    
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size=(3,3),activation='relu',strides=2,input_shape=(100,100,3),padding='same'))
    
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size=(3,3),strides=1,activation='relu',padding='valid'))
              
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size=(3,3),activation='relu',padding='valid'))
    model.add(MaxPooling2D())
    model.add(Dropout(.30))
    
    model.add(BatchNormalization())
    model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
              
    model.add(BatchNormalization())
    model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
    model.add(MaxPooling2D())
    
    model.add(BatchNormalization())
    model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
    model.add(Dropout(.20))

    model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
    model.add(MaxPooling2D())
    #model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
    model.add(Flatten())
    model.add(Dense(256,activation='relu'))
    model.add(Dense(128,activation='relu'))
    model.add(Dropout(.30))
    model.add(Dense(256,activation='relu'))
    model.add(Dense(128,activation='relu'))
    model.add(Dropout(.20))
    model.add(Dense(256,activation='relu'))
    model.add(Dense(128,activation='relu'))
    model.add(Dense(128,activation='relu'))
    model.add(Dense(120,activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,optimizer='adam',metrics=['accuracy'])
    
    YCatTrain=keras.utils.to_categorical(Y[train])
    
    h=model.fit(train_data1[train], YCatTrain,
          batch_size=128,
          epochs=10,
          validation_split = 0.2)
    
    YCatTest=keras.utils.to_categorical(Y[test])
    
    scores = model.evaluate(train_data1[test], YCatTest, verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    H.append(h)

In [ ]:
#model.compile(loss=keras.losses.categorical_crossentropy,optimizer='adam',metrics=['accuracy'])

In [ ]:
'''H=model.fit(X_train, y_train,
          batch_size=128,
          epochs=50,
          validation_split = 0.2)'''

In [ ]:
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
N = 100
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.title("Training Loss")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper left")
#plt.savefig(args["plot"])

In [ ]:
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper left")

In [ ]:
reversebreed={}
num=0
for item in range(120):
    reversebreed[item]=dogs[num]
    num+=1
#labels.breed=[reversebreed[item] for item in labels.breed]

In [ ]:
img_paths_test = "../input/test"

In [ ]:
testimgs = [load_img(img_paths_test+"/"+filename, target_size=(100, 100)) for filename in os.listdir(img_paths_test)]
test_img_array = np.array([img_to_array(img) for img in testimgs])   

In [ ]:
test_img_array.shape

In [ ]:
preds=model.predict_proba(test_img_array,batch_size=64)

In [ ]:
probDF=pd.DataFrame(preds)

In [ ]:
probDF.rename(reversebreed,axis=1,inplace=True)

In [ ]:
probDF=probDF.reindex_axis(sorted(probDF.columns), axis=1)

In [ ]:
probDF.head()

In [ ]:
imgnames=[]
for filename in os.listdir(img_paths_test):
    imgnames.append(filename.split('.')[0])

In [ ]:
imgnames=pd.DataFrame(imgnames)

In [ ]:
submission=imgnames.join(probDF)

In [ ]:
submission.rename(columns={0:"id"},inplace=True)

In [ ]:
submission.to_csv("Submission.csv",sep=',',encoding='utf-8',index=False)

In [ ]:
'''for pred in preds:
        top_indices = pred.argsort()[-3:][::-1]
        result = [ (pred[i],) for i in top_indices]
        result.sort(key=lambda x: x[0], reverse=True)
        print(result)'''

In [ ]:
#model.save("dogbreedmodel1.h5")